# FVG + Structure Verification

**IRS entry setup**: Price retraces to a **HTF Fair Value Gap** (POI), then a **1m structural event** (BOS / CHoCH / CISD) confirms directional commitment.

- **Bullish setup**: Price enters a bullish HTF FVG -> 1m bullish BOS/CHoCH/CISD fires within the zone
- **Bearish setup**: Price enters a bearish HTF FVG -> 1m bearish BOS/CHoCH/CISD fires within the zone

## Usage
1. Set `START_DATE`, `END_DATE` and `HTF_TF` in the config cell below
2. Run all cells
3. Use the **TF buttons** on the chart to switch candlestick granularity
4. Use Plotly's zoom / range selector to navigate
5. Change `SIGNAL_INDEX` in the drill-down cell to inspect individual matches

## Matching criteria ("inside an FVG")
- **Direction**: event direction == FVG direction
- **Time**: after FVG creation, before FVG mitigation/inversion/expiry
- **Price**: event level (broken_level or CISD level) within [FVG bottom, FVG top]

In [ ]:
import sys
sys.path.insert(0, '..')

from collections import Counter, defaultdict

import pandas as pd
import numpy as np
import plotly.graph_objects as go

from data.loader import load_instrument, detect_gaps
from data.resampler import resample
from concepts.fvg import detect_fvg, track_fvg_lifecycle
from concepts.structure import detect_bos_choch, detect_cisd

print('All imports OK')

In [ ]:
# ============================================================
# CONFIGURATION - edit these and re-run all cells below
# ============================================================

# Date range (avoid New Year / holiday gaps)
# Good ranges: '2024-09-10' to '2024-09-25' (trend), '2024-07-08' to '2024-07-22' (volatile)
START_DATE = '2025-01-06'
END_DATE   = '2025-01-17'

# HTF timeframe for FVG detection
HTF_TF = '15m'

# FVG parameters
FVG_MIN_GAP_PCT = 0.0003   # ~3.5pts on NAS100 at 11500
MITIGATION_MODE = 'close'
MAX_AGE_BARS    = 192      # HTF bars before FVG expires

# 1m structure detection
SWING_LENGTH_1M = 3        # Shorter swing for 1m (more sensitive)
CLOSE_BREAK     = True     # Require candle close for structure breaks

# Display
DEFAULT_TF_VIEW  = '1m'    # Which TF candles to show initially
SHOW_UNMATCHED   = False   # Show unmatched BOS/CHoCH as grey dots
ALL_TFS = ['1m', '5m', '15m', '30m', '1H', '4H', '1D']

print(f'Config: {START_DATE} -> {END_DATE}, HTF={HTF_TF}, LTF=1m')

In [ ]:
# ============================================================
# LOAD DATA + SLICE BY DATE
# ============================================================

def slice_by_date(df, start, end):
    """Filter DataFrame to [start, end] using the 'time' column."""
    s = pd.Timestamp(start, tz='UTC')
    e = pd.Timestamp(end, tz='UTC') + pd.Timedelta(days=1) - pd.Timedelta(seconds=1)
    mask = (df['time'] >= s) & (df['time'] <= e)
    return df[mask].copy()

print('Loading NAS100 1m data...')
df_1m_full = load_instrument('NAS100')
print(f'Full dataset: {len(df_1m_full):,} rows ({df_1m_full["time"].iloc[0]} to {df_1m_full["time"].iloc[-1]})')

# Slice to date range
df_1m = slice_by_date(df_1m_full, START_DATE, END_DATE)
print(f'\nSliced: {len(df_1m):,} 1m bars')
print(f'Actual range: {df_1m["time"].iloc[0]} to {df_1m["time"].iloc[-1]}')

# Check for gaps
gaps = detect_gaps(df_1m)
if len(gaps) > 0:
    big_gaps = gaps[gaps['gap_minutes'] > 120]  # > 2 hours (normal overnight gaps are expected)
    if len(big_gaps) > 0:
        print(f'\nWARNING: {len(big_gaps)} large gaps (>2h) found:')
        print(big_gaps.nlargest(5, 'gap_minutes').to_string(index=False))
    else:
        print('No unusual gaps (overnight gaps are normal).')
else:
    print('No gaps detected.')

# Resample to all timeframes for chart TF switching
print(f'\nResampling to all timeframes...')
dfs_by_tf = {}
for tf in ALL_TFS:
    dfs_by_tf[tf] = resample(df_1m, tf)
    print(f'  {tf}: {len(dfs_by_tf[tf]):,} bars')

df_htf = dfs_by_tf[HTF_TF]
print(f'\nHTF for FVG detection: {HTF_TF} ({len(df_htf):,} bars)')

In [ ]:
# ============================================================
# DETECT HTF FVGs + LIFECYCLE
# ============================================================

def build_lifecycle_with_timestamps(lifecycle, df_htf):
    """Convert lifecycle creation_index/end_index to actual timestamps."""
    records = []
    for lc in lifecycle:
        creation_idx = lc['creation_index']
        end_idx = lc['end_index']
        if creation_idx not in df_htf.index:
            continue
        creation_time = df_htf.loc[creation_idx, 'time']
        end_time = df_htf.loc[end_idx, 'time'] if end_idx in df_htf.index else df_htf['time'].iloc[-1]
        records.append({**lc, 'creation_time': creation_time, 'end_time': end_time})
    if not records:
        return pd.DataFrame()
    return pd.DataFrame(records)

# Detect FVGs
fvgs = detect_fvg(df_htf, min_gap_pct=FVG_MIN_GAP_PCT, join_consecutive=True)
lifecycle = track_fvg_lifecycle(df_htf, fvgs, mitigation_mode=MITIGATION_MODE, max_age_bars=MAX_AGE_BARS)
lc_df = build_lifecycle_with_timestamps(lifecycle, df_htf)

print(f'{HTF_TF} FVGs detected: {len(fvgs)}')
if len(fvgs) > 0:
    print(f'  Bullish: {(fvgs["direction"] == 1).sum()}')
    print(f'  Bearish: {(fvgs["direction"] == -1).sum()}')
if len(lc_df) > 0:
    statuses = Counter(str(s) for s in lc_df['status'])
    print(f'  Lifecycle: {dict(statuses)}')

In [ ]:
# ============================================================
# DETECT 1m STRUCTURE EVENTS + MATCH TO FVGs
# ============================================================

def find_events_in_fvgs(df_1m, lc_df, swing_length=3, close_break=True):
    """
    Find 1m BOS/CHoCH/CISD events inside HTF FVG zones.
    
    Three-gate filter:
      1. Direction: event.direction == fvg.direction
      2. Time: fvg.creation_time < event_time <= fvg.end_time
      3. Price: fvg.bottom <= event_level <= fvg.top
    
    Returns: (matches, bos_choch_df, cisd_df)
    """
    # Detect 1m structure events (once on full slice — preserves trend context)
    bos_choch_df = detect_bos_choch(df_1m, swing_length=swing_length, close_break=close_break)
    cisd_df = detect_cisd(df_1m)
    
    if len(lc_df) == 0:
        return [], bos_choch_df, cisd_df
    
    matches = []
    
    # Match BOS/CHoCH events
    for _, ev in bos_choch_df.iterrows():
        broken_idx = ev['broken_index']
        if broken_idx not in df_1m.index:
            continue
        ev_time = df_1m.loc[broken_idx, 'time']
        ev_level = ev['broken_level']
        ev_dir = ev['direction']
        ev_type = 'BOS' if 'BOS' in str(ev['type']) else 'CHoCH'
        
        mask = (
            (lc_df['direction'] == ev_dir) &
            (lc_df['creation_time'] < ev_time) &
            (lc_df['end_time'] >= ev_time) &
            (lc_df['bottom'] <= ev_level) &
            (lc_df['top'] >= ev_level)
        )
        for _, fvg in lc_df[mask].iterrows():
            matches.append({
                'signal_type': ev_type, 'direction': ev_dir,
                'event_time': ev_time, 'event_level': ev_level,
                'fvg_top': fvg['top'], 'fvg_bottom': fvg['bottom'],
                'fvg_midpoint': fvg['midpoint'], 'fvg_direction': fvg['direction'],
                'fvg_creation_time': fvg['creation_time'],
                'fvg_end_time': fvg['end_time'],
                'fvg_status': str(fvg['status']),
            })
    
    # Match CISD events
    for _, ev in cisd_df.iterrows():
        trigger_idx = ev['trigger_index']
        if trigger_idx not in df_1m.index:
            continue
        ev_time = df_1m.loc[trigger_idx, 'time']
        ev_level = ev['level']
        ev_dir = ev['direction']
        
        mask = (
            (lc_df['direction'] == ev_dir) &
            (lc_df['creation_time'] < ev_time) &
            (lc_df['end_time'] >= ev_time) &
            (lc_df['bottom'] <= ev_level) &
            (lc_df['top'] >= ev_level)
        )
        for _, fvg in lc_df[mask].iterrows():
            matches.append({
                'signal_type': 'CISD', 'direction': ev_dir,
                'event_time': ev_time, 'event_level': ev_level,
                'fvg_top': fvg['top'], 'fvg_bottom': fvg['bottom'],
                'fvg_midpoint': fvg['midpoint'], 'fvg_direction': fvg['direction'],
                'fvg_creation_time': fvg['creation_time'],
                'fvg_end_time': fvg['end_time'],
                'fvg_status': str(fvg['status']),
            })
    
    return matches, bos_choch_df, cisd_df


print('Detecting 1m structure events...')
matches, bos_choch_all, cisd_all = find_events_in_fvgs(
    df_1m, lc_df, swing_length=SWING_LENGTH_1M, close_break=CLOSE_BREAK
)

bos_count = (bos_choch_all['type'].astype(str).str.contains('BOS')).sum() if len(bos_choch_all) else 0
choch_count = (bos_choch_all['type'].astype(str).str.contains('CHOCH')).sum() if len(bos_choch_all) else 0
print(f'1m events: {bos_count} BOS, {choch_count} CHoCH, {len(cisd_all)} CISD')
print(f'\n==> Matched signals (1m event inside {HTF_TF} FVG): {len(matches)}')

if matches:
    by_type = defaultdict(lambda: {'long': 0, 'short': 0})
    for m in matches:
        side = 'long' if m['direction'] == 1 else 'short'
        by_type[m['signal_type']][side] += 1
    for t, counts in by_type.items():
        print(f'    {t}: {counts["long"]} long / {counts["short"]} short')

In [ ]:
# ============================================================
# MAIN CHART WITH TF SWITCHING BUTTONS
# ============================================================

# Marker styles: (symbol, color, size)
SIGNAL_STYLES = {
    ('BOS', 1):   ('diamond',       '#00bfff', 14),
    ('BOS', -1):  ('diamond',       '#ff4444', 14),
    ('CHoCH', 1): ('star',          '#00ff88', 14),
    ('CHoCH', -1):('star',          '#ff8800', 14),
    ('CISD', 1):  ('triangle-up',   '#ffff00', 12),
    ('CISD', -1): ('triangle-down', '#ff88ff', 12),
}

fig = go.Figure()

# --- Step 1: Add candlestick traces for each TF ---
tf_trace_indices = {}  # Map TF name -> trace index
for i, tf in enumerate(ALL_TFS):
    df_tf = dfs_by_tf[tf]
    visible = (tf == DEFAULT_TF_VIEW)
    fig.add_trace(go.Candlestick(
        x=df_tf['time'], open=df_tf['open'], high=df_tf['high'],
        low=df_tf['low'], close=df_tf['close'],
        name=tf, visible=visible,
        increasing_line_color='#26a69a', decreasing_line_color='#ef5350',
    ))
    tf_trace_indices[tf] = i

n_tf_traces = len(ALL_TFS)

# --- Step 2: Add signal marker traces ---
grouped = defaultdict(lambda: {'x': [], 'y': []})
for m in matches:
    key = (m['signal_type'], m['direction'])
    grouped[key]['x'].append(m['event_time'])
    grouped[key]['y'].append(m['event_level'])

marker_trace_start = len(fig.data)
for key, data in grouped.items():
    sym, col, sz = SIGNAL_STYLES.get(key, ('circle', 'white', 10))
    side = 'LONG' if key[1] == 1 else 'SHORT'
    fig.add_trace(go.Scatter(
        x=data['x'], y=data['y'],
        mode='markers', name=f'{key[0]} {side}',
        marker=dict(symbol=sym, color=col, size=sz, line=dict(color='white', width=1)),
        visible=True,
    ))

# Optional: unmatched events as grey dots
if SHOW_UNMATCHED and len(bos_choch_all) > 0:
    matched_times = {m['event_time'] for m in matches}
    unm_x, unm_y = [], []
    for _, ev in bos_choch_all.iterrows():
        idx = ev['broken_index']
        if idx not in df_1m.index:
            continue
        t = df_1m.loc[idx, 'time']
        if t not in matched_times:
            unm_x.append(t)
            unm_y.append(ev['broken_level'])
    if unm_x:
        fig.add_trace(go.Scatter(
            x=unm_x, y=unm_y, mode='markers',
            name='BOS/CHoCH (unmatched)',
            marker=dict(symbol='circle', color='rgba(128,128,128,0.3)', size=5),
            visible=True,
        ))

n_total_traces = len(fig.data)
n_overlay_traces = n_total_traces - n_tf_traces

# --- Step 3: Add FVG zone rectangles ---
if len(lc_df) > 0:
    for _, fvg in lc_df.iterrows():
        status = str(fvg['status'])
        alpha = 0.12
        if fvg['direction'] == 1:
            fill = f'rgba(0,200,0,{alpha})'
            border = f'rgba(0,200,0,{alpha * 3:.2f})'
        else:
            fill = f'rgba(200,0,0,{alpha})'
            border = f'rgba(200,0,0,{alpha * 3:.2f})'
        if 'FULLY_FILLED' in status or 'INVERTED' in status:
            fill = f'rgba(128,128,128,{alpha * 0.6:.2f})'
            border = f'rgba(128,128,128,{alpha * 1.5:.2f})'
        
        fig.add_shape(
            type='rect',
            x0=fvg['creation_time'], x1=fvg['end_time'],
            y0=fvg['bottom'], y1=fvg['top'],
            fillcolor=fill, line=dict(color=border, width=1),
        )
        # CE midpoint line
        if 'FULLY_FILLED' not in status and 'INVERTED' not in status:
            fig.add_shape(
                type='line',
                x0=fvg['creation_time'], x1=fvg['end_time'],
                y0=fvg['midpoint'], y1=fvg['midpoint'],
                line=dict(color=border, width=1, dash='dot'),
            )

# --- Step 4: TF switching buttons via updatemenus ---
buttons = []
for tf in ALL_TFS:
    # Visibility: one TF candle trace ON, all others OFF, all overlay traces ON
    vis = [False] * n_tf_traces + [True] * n_overlay_traces
    vis[tf_trace_indices[tf]] = True
    buttons.append(dict(
        label=tf,
        method='update',
        args=[{'visible': vis}],
    ))

fig.update_layout(
    title=f'NAS100 | {HTF_TF} FVGs + 1m entry signals | {START_DATE} to {END_DATE} | {len(matches)} matches',
    height=800,
    template='plotly_dark',
    xaxis_rangeslider_visible=False,
    showlegend=True,
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01,
                bgcolor='rgba(0,0,0,0.5)', bordercolor='grey', borderwidth=1),
    updatemenus=[
        dict(
            type='buttons',
            direction='right',
            x=0.0, xanchor='left',
            y=1.12, yanchor='top',
            buttons=buttons,
            showactive=True,
            active=ALL_TFS.index(DEFAULT_TF_VIEW),
            font=dict(size=11),
            bgcolor='rgba(40,40,40,0.8)',
            bordercolor='grey',
        ),
    ],
    # Range selector for zoom
    xaxis=dict(
        rangeselector=dict(
            buttons=[
                dict(count=4, label='4H', step='hour', stepmode='backward'),
                dict(count=1, label='1D', step='day', stepmode='backward'),
                dict(count=3, label='3D', step='day', stepmode='backward'),
                dict(count=7, label='1W', step='day', stepmode='backward'),
                dict(step='all', label='All'),
            ],
            y=1.06,
        ),
    ),
)

fig.show()
print(f'\nChart: {n_tf_traces} TF traces + {n_overlay_traces} overlay traces + {len(fig.layout.shapes)} FVG shapes')

In [ ]:
# ============================================================
# RESULTS TABLE
# ============================================================

if matches:
    df_results = pd.DataFrame(matches)
    df_results['side'] = df_results['direction'].map({1: 'LONG', -1: 'SHORT'})
    df_results = df_results.sort_values('event_time')
    
    # Format timestamps
    for col in ['event_time', 'fvg_creation_time', 'fvg_end_time']:
        df_results[col + '_str'] = df_results[col].dt.strftime('%Y-%m-%d %H:%M')
    
    display_df = df_results[[
        'event_time_str', 'side', 'signal_type',
        'event_level', 'fvg_bottom', 'fvg_top', 'fvg_midpoint',
        'fvg_creation_time_str', 'fvg_end_time_str', 'fvg_status',
    ]].rename(columns={
        'event_time_str': 'time', 'fvg_creation_time_str': 'fvg_created',
        'fvg_end_time_str': 'fvg_ended',
    }).reset_index(drop=True)
    
    print(f'Total matched signals: {len(df_results)}')
    print(f'  LONG:  {(df_results["direction"] == 1).sum()}')
    print(f'  SHORT: {(df_results["direction"] == -1).sum()}')
    print()
    display(display_df)
else:
    df_results = pd.DataFrame()
    print('No matched signals found.')
    print('Try: wider date range, different HTF, lower FVG_MIN_GAP_PCT, or SWING_LENGTH_1M=2')

In [ ]:
# ============================================================
# SIGNAL DRILL-DOWN - change SIGNAL_INDEX and re-run this cell
# ============================================================

SIGNAL_INDEX = 0  # Change this to inspect different matches

if matches and SIGNAL_INDEX < len(matches):
    m = matches[SIGNAL_INDEX]
    event_time = m['event_time']
    side = 'LONG' if m['direction'] == 1 else 'SHORT'
    
    # Zoom window: +/- 4 hours around the signal
    zoom_start = event_time - pd.Timedelta(hours=4)
    zoom_end = event_time + pd.Timedelta(hours=4)
    df_zoom = slice_by_date(df_1m_full,
                            zoom_start.strftime('%Y-%m-%d %H:%M'),
                            zoom_end.strftime('%Y-%m-%d %H:%M'))
    
    title = f"Signal #{SIGNAL_INDEX}: {m['signal_type']} {side} @ {event_time.strftime('%Y-%m-%d %H:%M')}"
    
    fig_d = go.Figure()
    fig_d.add_trace(go.Candlestick(
        x=df_zoom['time'], open=df_zoom['open'], high=df_zoom['high'],
        low=df_zoom['low'], close=df_zoom['close'], name='1m',
        increasing_line_color='#26a69a', decreasing_line_color='#ef5350',
    ))
    
    # FVG zone rectangle
    fvg_color = 'rgba(0,200,0,0.15)' if m['direction'] == 1 else 'rgba(200,0,0,0.15)'
    fvg_border = 'rgba(0,200,0,0.5)' if m['direction'] == 1 else 'rgba(200,0,0,0.5)'
    fig_d.add_shape(
        type='rect',
        x0=m['fvg_creation_time'], x1=m['fvg_end_time'],
        y0=m['fvg_bottom'], y1=m['fvg_top'],
        fillcolor=fvg_color, line=dict(color=fvg_border, width=1),
    )
    # CE midpoint
    fig_d.add_hline(y=m['fvg_midpoint'], line_color='rgba(200,200,200,0.5)',
                    line_dash='dot', annotation_text='CE')
    
    # Signal marker
    sym, col, sz = SIGNAL_STYLES.get((m['signal_type'], m['direction']), ('circle', 'white', 14))
    fig_d.add_trace(go.Scatter(
        x=[event_time], y=[m['event_level']],
        mode='markers+text',
        text=[f"{m['signal_type']} {side}"],
        textposition='top center',
        marker=dict(symbol=sym, color=col, size=18, line=dict(color='white', width=2)),
        name='Signal',
    ))
    
    fig_d.update_layout(
        title=title, height=600, template='plotly_dark',
        xaxis_rangeslider_visible=False, showlegend=True,
    )
    
    print(f'Signal #{SIGNAL_INDEX}:')
    print(f'  Type:       {m["signal_type"]} {side}')
    print(f'  Time:       {event_time}')
    print(f'  Level:      {m["event_level"]:.2f}')
    print(f'  FVG zone:   [{m["fvg_bottom"]:.2f}, {m["fvg_top"]:.2f}] (CE {m["fvg_midpoint"]:.2f})')
    print(f'  FVG status: {m["fvg_status"]}')
    print(f'  FVG alive:  {m["fvg_creation_time"]} -> {m["fvg_end_time"]}')
    fig_d.show()
else:
    print('No matches to drill down into. Run the analysis cells above first.')
    if matches:
        print(f'Valid SIGNAL_INDEX range: 0 to {len(matches)-1}')